In [1]:
# Imports
!pip install --upgrade tensorflow
!pip install tensorflow-gpu
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install datasets
!pip install tokenizers
!pip install transformers
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu
!pip install wget

!python -m wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!python -m wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)
     |████████████████████████████████| 394.3MB 44kB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 7.6MB 8.2MB/s 
     |████████████████████████████████| 184kB 8.0MB/s 
     |████████████████████████████████| 112kB 13.3MB/s 
     |████████████████████████████████| 245kB 13.5MB/s 
     |████████████████████████████████| 3.2MB 7.7MB/s 
     |████████████████████████████████| 1.9MB 8.6MB/s 
     |████████████████████████████████| 890kB 25.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6a398bdb60cc0a2cbace2747db9bfde1f6c4ee930e15a5fca98a2a6a7b1a2f3f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 8.6MB/s 
     |████████████████████████████████| 7

In [2]:
# Imports
import datasets
import pandas as pd
import csv

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import EncoderDecoderModel, BertTokenizer, BertTokenizerFast, DistilBertTokenizer, DistilBertTokenizerFast

In [3]:
# Test CUDA
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print("Device:", device)

Device: cuda


In [4]:
# Load german data
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)
path_corpus = path_drive + "/My Drive/Temp/Corpus/data_train.csv"

data_txt = []
data_ref = []

with open(path_corpus, "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_ALL)
    next(reader, None)

    for row in reader:
        data_txt.append(row[0])
        data_ref.append(row[1])

tuples = list(zip(data_txt, data_ref))
dataframe = pd.DataFrame(tuples, columns=["article", "highlights"])
german_data = datasets.arrow_dataset.Dataset.from_pandas(dataframe)

train_data = german_data.select(range(0, 85000))
val_data = german_data.select(range(85000, 95000))
test_data = german_data.select(range(95000, 100000))

Mounted at /content/drive


In [ ]:
# Load english data
'''
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]")
'''

'\ntrain_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")\nval_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")\ntest_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]")\n'

In [6]:
# Analyze corpus exploration
'''
df = pd.DataFrame(train_data)

text_list = []
summary_list = []

for index, row in df.iterrows():
    text = row["article"]
    summary = row["highlights"]
    text_list.append(len(text))
    summary_list.append(len(summary))
    
print(sum(text_list) / len(text_list))
print(sum(summary_list) / len(summary_list))
'''

                                                 article                                         highlights
0      Minghella war der Sohn italienisch-schottische...  Anthony Minghella, CBE war ein britischer Film...
1      Ende der 1940er Jahre wurde eine erste Auteur-...  Die Auteur-Theorie ist eine Filmtheorie und di...
2      Al Pacino, geboren in Manhattan, ist der Sohn ...  Alfredo James "Al" Pacino ist ein US-amerikani...
3      Der Name der Alkalimetalle leitet sich von dem...  Als Alkalimetalle werden die chemischen Elemen...
4      Die Arbeit ist bereits seit dem Altertum Gegen...  Das deutsche Arbeitsrecht ist ein Rechtsgebiet...
...                                                  ...                                                ...
84995  Nach dem Ende des Staates Österreich-Ungarn w...  Der Regionalwahlkreis Wien Süd ist ein Region...
84996  Nach dem Ende des Staates Österreich-Ungarn w...  Der Regionalwahlkreis Wien Nord-West ist ein R...
84997  Hermann Mayer absolvi

In [ ]:
# Explore corpus
'''
train_data.info.description
df = pd.DataFrame(train_data[:1])

# del df["id"]

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html()))
'''

,article,highlights
0,"Minghella war der Sohn italienisch-schottischer Eltern, die auf der Isle of Wight eine Fabrik für Eiscreme betrieben. Nach seinem Schulabschluss studierte er an der Universität Hull, wo er eine Zeit lang als Dozent tätig war. 1978 drehte er einen ersten Kurzfilm. Seit 1981 war er als Autor und Story Editor tätig. Er wurde mit Theaterstücken, Rundfunkhörspielen, der Fernsehserie ""Inspector Morse"" und vielen Drehbüchern für Film und Fernsehen bekannt. Er entwickelte die Drehbücher für die 1988 erfolgreich ausgestrahlte Fernsehserie The Storyteller von Muppets-Erfinder Jim Henson. Auch als Produzent war er erfolgreich, darunter für die Filme ""Der stille Amerikaner"", ""Die Dolmetscherin"" und ""Der Vorleser"", für den er 2008 posthum für den Oscar nominiert wurde. Gemeinsam mit seinem Freund und Kollegen Sydney Pollack gründete er die Produktionsfirma Mirage Enterprises. Der Regisseur Minghella galt als ein guter Schauspielerführer: Unter seiner Regie brachten es zahlreiche Darsteller zu Oscar-Nominierungen, zwei Schauspielerinnen erhielten die Auszeichnung als ""Beste Nebendarstellerin"": Juliette Binoche und Renée Zellweger . Gegen Ende seines Lebens kehrte Minghella zu seinen Anfängen im Radio und auf der Bühne zurück: 2006 wurde sein Hörspiel ""Eyes Down Looking"" mit Jude Law zu Ehren von Samuel Beckett auf BBC Radio 3 ausgestrahlt, ein Jahr zuvor hatte seine Inszenierung der Puccini-Oper Madame Butterfly in der English National Opera in London Premiere und wurde auch in der Nationaloper von Vilnius und in der Metropolitan Opera in New York gezeigt. Am Ende des Films ""Abbitte"" von Joe Wright hat er einen Kurzauftritt als Talkshow-Moderator neben Vanessa Redgrave. Seine letzte Arbeit als Drehbuchautor war das Skript für den Musical-Film ""Nine"" . Zu seinen letzten Regiearbeiten zählt der Pilotfilm zur Krimiserie ""Eine Detektivin für Botswana"" , den die BBC fünf Tage nach seinem Tod erstmals ausstrahlte. Minghella war mit der aus Hongkong stammenden Choreographin, Produzentin und Schauspielerin Carolyn Choa verheiratet. Der Ehe entstammen zwei Kinder, die in der Filmbranche tätig sind: Tochter Hannah Minghella in der Produktion und Sohn Max Minghella als Schauspieler . Die Tante Edana Minghella und der Onkel Dominic Minghella sind Drehbuchautoren. Minghella starb im Alter von 54 Jahren in einem Londoner Krankenhaus an inneren Blutungen infolge der Operation eines Tonsillenkarzinoms und eines Karzinoms im Nacken. 1984 erhielt Minghella den Londoner Kritikerpreis als meistversprechender junger Dramatiker, 1986 den Kritikerpreis für sein Stück ""Made in Bangkok"" als bestes Stück der Saison. 1997 erhielt er für ""Der englische Patient"" den Oscar in der Rubrik ""Beste Regie"", 1999 eine Oscar-Nominierung in der Kategorie ""Bestes adaptiertes Drehbuch"" für ""Der talentierte Mr. Ripley"", bei dem er auch Regie führte. 2001 wurde Minghella zum Commander of the British Empire ernannt. Von 2003 bis 2007 war er Präsident des British Film Institute. Seit 1997 trägt das Anthony Minghella Theatre auf der Isle of Wight seinen Namen.","Anthony Minghella, CBE war ein britischer Filmregisseur, Filmproduzent, Drehbuchautor, Dramatiker, Hörspiel-Autor, Theater- und Opern-Regisseur."


In [ ]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

In [ ]:
# Prepare data
encoder_max_length = 512
decoder_max_length = 128
batch_size = 4 # 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [ ]:
# Training data
train_data = train_data.shuffle()

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights"] # "id"
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

In [ ]:
# Validation data
val_data = val_data.shuffle()

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights"] # id
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

val_data = val_data.select(range(2000))

In [ ]:
# Load models
tf2tf = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased", tie_encoder_decoder=False)
tf2tf.save_pretrained("bert2bert_multilingual")
tf2tf = EncoderDecoderModel.from_pretrained("bert2bert_multilingual")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [ ]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

In [ ]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [ ]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# Load checkpoint
from google.colab import drive

path_drive = "/content/drive"
drive.mount(path_drive)

path_output = path_drive + "/My Drive/Temp/Models"
path_checkpoint = path_output + "/checkpoint-16000"

tf2tf = EncoderDecoderModel.from_pretrained(path_checkpoint)
tf2tf.to("cuda")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# Setup arguments
from seq2seq_training_args import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir=path_output, # "./"
    warmup_steps=1000,
    save_steps=2000,
    logging_steps=100,
    eval_steps=2000,
    save_total_limit=1,
    fp16=True
)

In [ ]:
# Start training
from seq2seq_trainer import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Runtime,Samples Per Second
2000,1.872500,2.565481,0.131300,0.269900,0.166400,1496.531800,1.336000
4000,1.744100,2.559587,0.139100,0.283800,0.175900,1472.201600,1.359000
6000,1.846100,2.519058,0.140900,0.281900,0.176500,1412.490700,1.416000
8000,2.312600,2.364513,0.141200,0.290900,0.178700,1475.179700,1.356000
10000,2.317000,2.332419,0.147400,0.297400,0.185200,1454.648100,1.375000
12000,2.421200,2.246138,0.146400,0.301200,0.185100,1486.992100,1.345000
14000,2.354100,2.206077,0.156900,0.312100,0.196100,1472.707800,1.358000
16000,2.338100,2.168354,0.157800,0.310600,0.196500,1442.568900,1.386000


In [ ]:
# Evaluate training
def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred_summary"] = output_str

    return batch

results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article"]
)

print(results[0]["pred_summary"])
print(results[0]["highlights"])
print("====================")

rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid

# tf2tf = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail").to("cuda")
# tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")


Das Museum Relandersgrund ist ein Museumsschiff der finnischen Marine. Es wurde 1888 und ist heute eines der Museumsschiffe der Welt. Das Museum ist eines von drei Museumsschiffen der Welt, die von der " Relander Grund AG " betrieben werden.
Das Feuerschiff Relandersgrund war ein finnisches Feuerschiff, das von 1888 bis 1914 im Schärenmeer bei Rauma positioniert war. Heute dient es als Restaurantschiff in Helsinki.


Score(precision=0.13773718758838316, recall=0.2310497425786467, fmeasure=0.1599690596019353)